In [21]:
# 导入所需的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import os
from sklearn.metrics import mean_squared_error

In [22]:
raw_train_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\train_data.csv")
raw_test_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\test_data.csv")
raw_dev_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\dev_data.csv")

In [27]:
# 删除第一列
train_data = raw_train_data.drop(raw_train_data.columns[0], axis=1)
dev_data = raw_dev_data.drop(raw_dev_data.columns[0], axis=1)
test_data = raw_test_data.drop(raw_test_data.columns[0], axis=1)
# 将值为0的数替换为NaN
train_data.replace(0, np.nan, inplace=True)

#填补缺失值
train_data['CCCT'] = train_data['CCCT'].fillna(train_data['CCCT'].mean())
train_data['CVCT'] = train_data['CVCT'].fillna(train_data['CVCT'].mean())
# 查找缺失值
# missing_values = train_data.isnull().sum()
# missing_values

t_data = test_data[test_data['SoH'] == -100]

# grouped_data = train_data.groupby("CS_Name")
# for group_name, group_data in grouped_data:
#     print("Group Name:", group_name)
#     print(group_data)

In [29]:
# 建立一个空的 DataFrame 来存储结果
results_df = pd.DataFrame(columns=['cycle', 'CS_Name', 'result'])

# 根据 CS_Name 列分组
for group_name, group_data in train_data.groupby('CS_Name'):
    # 划分特征和标签
    X_group = group_data.drop(columns=['SoH'])
    y_group = group_data['SoH']

# 数据预处理
# 可以根据需要进行特征选择、特征缩放、处理缺失值等操作

# 划分特征和标签
X_test = t_data.drop(columns=['SoH'])
# y_test = t_data['SoH']

# 建立空列表，用于存储每个组的模型
models = {}

# 根据 CS_Name 列分组
for group_name, group_data in train_data.groupby('CS_Name'):
    # 划分特征和标签
    X_group = group_data.drop(columns=['SoH'])
    X_group = X_group.drop(columns=['CS_Name'])
    y_group = group_data['SoH']

    # 建立XGB模型
    model = xgb.XGBRegressor(
        n_estimators=1000,  # 迭代次数
        max_depth=5,  # 树的最大深度
        learning_rate=0.1,  # 学习率
        objective='reg:squarederror',  # 损失函数
        random_state=42
    )

    # 训练模型
    model.fit(X_group, y_group)

    # 将训练好的模型存储到字典中
    models[group_name] = model

# 分组预测并计算RMSE
rmse_list = []

# 根据 CS_Name 列分组
for group_name, group_data in X_test.groupby('CS_Name'):
    # 获取对应组的模型
    model = models[group_name]
    group_data=group_data.drop(columns=['CS_Name'])
    # 对每个组进行预测
    y_pred_group = model.predict(group_data)


    # 获取该组对应的 cycle 列
    cycles = X_test[X_test['CS_Name'] == group_name]['cycle']

    # 将结果添加到结果 DataFrame 中
    group_results = pd.DataFrame({
        'cycle': cycles,
        'CS_Name': [group_name] * len(cycles),
        'result': y_pred_group
    })

    results_df = pd.concat([results_df, group_results], ignore_index=True)

# 根据 'Cycle' 和 'CS_Name' 列进行内连接
merged_df = pd.merge(results_df, raw_test_data[[ 'Unnamed: 0','cycle', 'CS_Name']],
                     on=['cycle', 'CS_Name'], how='inner')
# 删除 merged_df 中的 'Unnamed: 0' 列，并插入到第一列位置
unnamed_column = merged_df.pop('Unnamed: 0')
merged_df.insert(0, unnamed_column.name, unnamed_column)
# 将合并后的结果保存为新的 CSV 文件
merged_df.to_csv('merged_file6.csv', index=False)
#
#     # 计算每个组的 RMSE
#     rmse_group = np.sqrt(mean_squared_error(y_test.loc[group_data.index], y_pred_group))
#
#     # 打印每个组的 RMSE
#     print("RMSE for group", group_name, ":", rmse_group)
#
#     # 添加到 RMSE 列表中
#     rmse_list.append(rmse_group)
#
# # 计算总体 RMSE
# overall_rmse = np.mean(rmse_list)
# print("Overall Root Mean Square Error (RMSE):", overall_rmse)

In [ ]:


# 根据 CS_Name 列分组
for group_name, group_data in X_test.groupby('CS_Name'):
    # 获取对应组的模型
    model = models[group_name]

    # 删除 'CS_Name' 列
    group_data = group_data.drop(columns=['CS_Name'])

    # 对每个组进行预测
    y_pred_group = model.predict(group_data)

    # 获取该组对应的 cycle 列
    cycles = X_test[X_test['CS_Name'] == group_name]['cycle']

    # 将结果添加到结果 DataFrame 中
    group_results = pd.DataFrame({
        'Cycle': cycles,
        'CS_Name': [group_name] * len(cycles),
        'Result': y_pred_group
    })

    results_df = pd.concat([results_df, group_results], ignore_index=True)

# 将结果保存到 CSV 文件中
results_df.to_csv('group_predictions.csv', index=False)

In [11]:
raw_dev_data

,Unnamed: 0,cycle,capacity,SoH,resistance,CCCT,CVCT,CS_Name
0,0,765.0,0.577454,0.375810,0.114167,2654.376105,3390.160481,CS2_35
1,1,766.0,0.595849,0.421673,0.112446,2620.392385,3403.597546,CS2_35
2,2,767.0,0.605033,0.430860,0.110828,2605.829928,3408.253776,CS2_35
3,3,768.0,0.605038,0.430859,0.110928,2589.361623,NaN,CS2_35
4,4,769.0,0.605031,0.430855,0.112344,2693.484955,3589.658648,CS2_35
...,...,...,...,...,...,...,...,...
571,148,1010.0,0.366656,-100.000000,0.122835,1485.021686,3244.713049,CS2_38
572,149,1011.0,0.366665,-100.000000,0.123425,1400.679296,3255.944861,CS2_38
573,150,1012.0,0.357495,-100.000000,0.124349,1356.867640,3243.913667,CS2_38
574,151,1013.0,0.357480,-100.000000,0.123536,1327.852449,3242.275622,CS2_38


In [26]:
t_data

,cycle,capacity,SoH,resistance,CCCT,CVCT,CS_Name
105,735.0,0.660063,-100.0,0.106262,3226.117381,3189.414722,CS2_35
106,736.0,0.650903,-100.0,0.107884,3201.211427,3193.823261,CS2_35
107,737.0,0.641716,-100.0,0.109112,3210.305224,3161.853217,CS2_35
108,738.0,0.641700,-100.0,0.109408,3258.180260,3037.543877,CS2_35
109,739.0,0.632537,-100.0,0.108695,3217.883697,3142.181698,CS2_35
...,...,...,...,...,...,...,...
570,857.0,0.614230,-100.0,0.111245,3002.888352,3206.960871,CS2_38
571,858.0,0.605064,-100.0,0.111345,2961.199236,3206.492017,CS2_38
572,859.0,0.605071,-100.0,0.110633,2923.872113,3221.273252,CS2_38
573,860.0,0.605073,-100.0,0.110533,2907.340689,3222.194994,CS2_38
